Firstly you should have the "Azure AI Services" resource in your Azure resource group, and from this resource you should have the Azure Keys and Endpoints (For the above )

Step 1: Import Necessary Libraries

In [7]:
!pip install azure-ai-vision-imageanalysis==1.0.0b1
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.6 MB/s eta 0:00:00


In [9]:
# Import necessary libraries
import os
import time
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import fitz  # PyMuPDF
from google.colab import files

Step 2: Load Configuration Settings

In [10]:
## declare the required variables
ai_endpoint = "your_Azure_AI_Service_endpoint"
ai_key= 'your_Azure_AI_Service_key'

Step 3: Authenticate Azure AI Vision Client

In [11]:
# Authenticate Azure AI Vision client
cv_client = ImageAnalysisClient(
    endpoint=ai_endpoint,
    credential=AzureKeyCredential(ai_key)
)


Step 4: Define Function to Convert PDF to Images

In [13]:
def pdf_to_images(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        output_path = os.path.join(output_folder, f'page_{page_num + 1}.jpg')
        pix.save(output_path)
    return [os.path.join(output_folder, f'page_{i + 1}.jpg') for i in range(len(doc))]


Step 5: Define Function to Extract Text from Images

In [14]:
def GetTextRead(image_file):
    print('\n')
    # Open image file
    with open(image_file, "rb") as f:
        image_data = f.read()

    # Use Analyze image function to read text in image
    result = cv_client.analyze(
        image_data=image_data,
        visual_features=[VisualFeatures.READ]
    )

    # Extract text from the image
    text = ""
    if result.read is not None:
        for line in result.read.blocks[0].lines:
            text += f"{line.text}\n"
    return text


Step 6: Define Main Function to Process PDF

In [15]:
def main():
    global cv_client
    try:
        # Upload PDF file
        uploaded = files.upload()
        pdf_path = next(iter(uploaded))

        # Load PDF and convert to images
        output_folder = 'images'
        os.makedirs(output_folder, exist_ok=True)
        image_files = pdf_to_images(pdf_path, output_folder)

        # Extract text from each image and write to a text file
        with open('output_text.txt', 'w') as output_file:
            for image_file in image_files:
                text = GetTextRead(image_file)
                output_file.write(text + '\n')

        print('Text extraction complete. Results saved in output_text.txt')
    except Exception as ex:
        print(ex)


In [16]:
if __name__ == "__main__":
    main()


Saving NLP_Theory.pdf to NLP_Theory.pdf






























Text extraction complete. Results saved in output_text.txt
